# Sorting Robot with a Twist
## Project Checklist
### Core Tasks
- [x] Create all the necessary classes and methods to create reusable code.
- [x] Create an algorithm to identify and sort objects (e.g., colored cubes) into designated bins based on their color.
- [x] Use the Panda robot manipulator and assume the color of the objects is known.
### For Each Detected Object:
- [x] Compute the picking position (pick) and the placing position (place) based on the object’s color.
- [x] Use inverse kinematics to obtain the robot configurations (q_pick and q_place).

### Control Strategies
- [ ] Implement joint-space trajectory interpolation: move from pick to place positions using interpolated joint trajectories between q_pick and q_place.
- [x] Implement resolved-rate motion control: use Cartesian velocity control to reach pick and place poses smoothly.
### Analysis
- [ ] Plot and compare trajectories for both strategies (joint and Cartesian space).
- [ ] Analyze end-effector path smoothness.
- [ ] Analyze execution time.
- [ ] Analyze sorting accuracy.
### Extra Steps
- [ ] Mock a camera so that the robot has to find the location and color of the spheres based on visual input. (NOT FEASIBLE)
- [ ] Putting cylinders between the berries and the boxes and create vectors of repulsion around these objects, this would make the robot sense the obstacles.
## Aesthetic Modifications
- [ ] Create a matrix of coordinates in the boxes to place the berries in the right position
#### Sensing Approaches
1. [ ] Use OpenCV to process the camera image, using HSV color masking to find the pixel coordinates of each berry. (NOT FEASIBLE)
2. [ ] Use a Convolutional Neural Network (CNN) that takes the raw camera image as direct input and is trained to learn its own policy for “seeing” the berries. (NOT FEASIBLE)

## key observations during the presentation
1. Add some maths to the notebook to state clearly what has been done
2. Remember to add numbers to the slides such that the professor can go back to the slide
3. Keep the slide as minimal as possible, not much text
4. Prepare a backup video for the presentation
5. 

# Introduction
In this notebook, we present our project on developing a sorting robot for berries. The robot is designed to identify and sort different types of berries based on their color, size, and quality using computer vision and machine learning techniques.

In [ ]:

from email import errors
from shutil import move
from matplotlib.pylab import box
import swift
import roboticstoolbox as rtb
import spatialgeometry as sg
import spatialmath as sm
import numpy as np
import time

red = [1, 0, 0]
green = [0, 1, 0]
blue = [0, 0, 1]
orange = [1, 0.5, 0]
colors = [red, green, blue]

light_red = [1, 0.8, 0.8]
light_green = [0.8, 1, 0.8]
light_blue = [0.8, 0.8, 1]


class Berry:
    def __init__(self, radius, table_position_x, table_position_y, table_position_z, table_height, table_depth, table_width):
        self.table_position_z = table_position_z
        self.table_height = table_height
        self.radius = radius
        self.random_color = colors[np.random.choice(len(colors))]
        self.is_picked = False
        
        # Modified positioning to keep berries in front of robot
        # Robot is at (0, -0.5, 0), so berries should be at positive Y relative to robot
        min_x = table_position_x - table_depth/2 + self.radius + 0.1
        max_x = table_position_x + table_depth/2 - self.radius - 0.1
        
        # Keep berries in front of robot (closer to table center, away from robot)
        min_y = table_position_y - table_width/4 + self.radius  # Reduced range
        max_y = table_position_y + table_width/4 - self.radius  # Reduced range
        
        random_x_position_on_table = np.random.uniform(min_x, max_x)
        random_y_position_on_table = np.random.uniform(min_y, max_y)
        self.std_pose_z = table_position_z + table_height/2 + self.radius
        random_position = sm.SE3(random_x_position_on_table, random_y_position_on_table, self.std_pose_z)
        self._sphere = sg.Sphere(radius=radius, pose=random_position, color=self.random_color)
        
    @property
    def pose(self):
        return sm.SE3(self._sphere.T)  # Return an SE3 object from the sphere's transformation matrix

    @property
    def swift(self):
        return self._sphere
    
    def eat(self):
        self._sphere.T = sm.SE3(0, 0, 1000).A  # Use .A to get the 4x4 numpy array
    
    def set_position(self, position):
        self._sphere.T = position.A  # Use .A to get the 4x4 numpy array from SE3

    def pick_up(self):
        self.is_picked = True

    def drop_at_position(self, position):
        self.is_picked = False
        drop_position = sm.SE3(position.x, position.y, self.std_pose_z - 0.03) # better into the box
        self.set_position(drop_position)

    def update_position_to_end_effector(self, end_effector_pose):
        if self.is_picked:
            berry_offset = sm.SE3(0, 0, -self.radius )
            new_position = end_effector_pose * berry_offset
            self.set_position(new_position)

    def get_drop_position_by_color(self, box1_pos, box2_pos, box3_pos):
        if np.array_equal(self.random_color, red):
            return box1_pos
        elif np.array_equal(self.random_color, green):
            return box2_pos
        elif np.array_equal(self.random_color, blue):
            return box3_pos
        else:
            return None

class Box:
    def __init__(self, width, depth, height, position, color=[0.9, 0.9, 0.9], wall_thickness=0.02):
        self.width = width
        self.depth = depth
        self.height = height
        self.position = position
        self.color = color
        self.wall_thickness = wall_thickness
        self.boxes = []
        bottom = sg.Cuboid(scale=[width, depth, wall_thickness],pose=sm.SE3(position.x, position.y, position.z - height/2 + wall_thickness/2), color=color)
        self.boxes.append(bottom)
        front = sg.Cuboid(scale=[width, wall_thickness, height - 2*wall_thickness], pose=sm.SE3(position.x, position.y - depth/2 + wall_thickness/2, position.z),color=color)
        self.boxes.append(front)
        back = sg.Cuboid(scale=[width, wall_thickness, height - 2*wall_thickness], pose=sm.SE3(position.x, position.y + depth/2 - wall_thickness/2, position.z), color=color)
        self.boxes.append(back)
        left = sg.Cuboid(scale=[wall_thickness, depth - 2*wall_thickness, height - 2*wall_thickness],pose=sm.SE3(position.x - width/2 + wall_thickness/2, position.y, position.z),color=color)
        self.boxes.append(left)
        right = sg.Cuboid(scale=[wall_thickness, depth - 2*wall_thickness, height - 2*wall_thickness], pose=sm.SE3(position.x + width/2 - wall_thickness/2, position.y, position.z), color=color)
        self.boxes.append(right)

    @property
    def swift(self):
        return self.boxes
    
class Table:    
    def __init__(self, x, y, z, depth, width, height, color):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
        self.depth = float(depth)
        self.width = float(width)
        self.height = float(height) 
        self._cuboid = sg.Cuboid(scale=[self.depth, self.width, self.height], pose=sm.SE3(self.x, self.y, self.z), color=color)
    
    @property
    def table_position_z(self):
        return float(self.z)
    
    @property
    def table_height(self):
        return float(self.height)
    
    @property
    def swift(self):
        return self._cuboid
    
    @swift.setter
    def swift(self, swift):
        self._cuboid = swift

    
class Cylinder:
    def __init__(self, radius, height, position, color):
        self.radius = radius
        self.height = height
        self.position = position
        self._cylinder = sg.Cylinder(radius=radius, height=height, pose=position, color=color)
    
    @property
    def swift(self):
        return self._cylinder
    
    

class Robot:
    def __init__(self, base_position=sm.SE3(0,0,0)):
        self.dt=0.01 # time step
        self.error = np.ones((3,1)) # initial error
        self.gain = 1.5 # control gain
        self.dumping_factor = 0.15
        self.picked_berry = None
        self.base_position = base_position
        self._panda = rtb.models.Panda()
        self._panda.base = base_position
        self.rest_pose = np.array([np.pi/4, -np.pi/6, 0, -2*np.pi/3, 0, np.pi/2, np.pi/4])
        self._panda.q = self.rest_pose

    @property
    def swift(self):
        return self._panda

    @property
    def end_effector_pose(self):
        return self._panda.fkine(self._panda.q) # returns an SE3 object

    def move_to_target_resolved_rate_of_motion(self, target): # with resolved rate of motion control
        self.error = target.t - self._panda.fkine(self._panda.q).t
        J = self._panda.jacob0(self._panda.q)[0:3, :]
        J_damped_pinv = J.T @ np.linalg.inv(J @ J.T + self.dumping_factor * np.eye(3))
        q_dot = self.gain * J_damped_pinv @ self.error
        self._panda.qd = q_dot
        if self.picked_berry:
            self.picked_berry.update_position_to_end_effector(self.end_effector_pose)

    def move_target_joint_space(self,target, trajectory_time=1.0):
        #first we get the target joint configuration using inverse cinematics
        q_target = self._panda.ikine_LM(target).q # inverse kinematics to get target joint configuration
        q_start = self._panda.q.copy() #we save the initial joint configuration
        steps = int(trajectory_time / self.dt) #number of steps for the trajectory
        trajectory = rtb.jtraj(q_start, q_target, steps) #joint space trajectory

        errors_history = []
        for i, q in enumerate(trajectory.q):
            self._panda.q = q #move robot to the next joint configuration in the trajectory

            current_pose = self._panda.fkine(self._panda.q)
            self.error = target.t - current_pose.t
            current_error = np.linalg.norm(self.error)
            errors_history.append(current_error)

            if self.picked_berry:
                self.picked_berry.update_position_to_end_effector(self.end_effector_pose)

            yield current_error
            
        return errors_history
    

    def pick_berry(self, berry):
        self.picked_berry = berry
        berry.pick_up()
    
    def drop_berry_at(self, drop_position):
        if self.picked_berry:
            self.picked_berry.drop_at_position(drop_position)
            self.picked_berry = None
    
    def set_robot_position(self, position):
        self._panda.q = position

    def reset_error(self):
        self.error = np.ones((3,1))

    def has_converged(self, errors_history, threshold=0.001, window=10):
        if len(errors_history) < window:
            return False
        recent_errors = errors_history[-window:]
        error_change = np.abs(recent_errors[-1] - recent_errors[0])
        return error_change < threshold

class Environment:
    def __init__(self):
        self.swift_env = swift.Swift()
        self.swift_env.launch(realtime=True,comms="rtc",browser="notebook")
        self.berries = []

    def add_multiple_objects_swift(self, objects):
        for obj in objects:
            self.swift_env.add(obj)



def resolved_rate_of_motion_sorting_task():

    env = Environment()
    movement_precision = 0.01
    table_height = 0.08
    table_width = 0.6
    table_depth = 1
    table_position_x = 0.0
    table_position_y = 0.0
    table_position_z = 0.25
    berry_radius = 0.02
    
    robot_base_x = 0.0
    robot_base_y = -0.5
    robot_base_z = 0.0
    
    box1_position_y = 0.0
    wall_thickness = 0.02
    box1_position_z = table_position_z + table_height + wall_thickness / 2 - 0.01
    box_width = 0.12
    box_depth = 0.12
    box_height = 0.08
    
    table1 = Table(table_position_x, table_position_y, table_position_z, table_depth, table_width, table_height, orange)
    cm_from_table_edge_to_box = table1.depth/2 + box_depth/2 - box_width
    box1_position_x = table_position_x - cm_from_table_edge_to_box
    box_red = Box(box_width, box_depth, box_height, sm.SE3(box1_position_x , box1_position_y - box_width - 0.03, box1_position_z), color=light_red, wall_thickness=wall_thickness)
    box_green = Box(box_width, box_depth, box_height, sm.SE3(box1_position_x , box1_position_y, box1_position_z), color=light_green, wall_thickness=wall_thickness)
    box_blue = Box(box_width, box_depth, box_height, sm.SE3(box1_position_x , box1_position_y + box_width + 0.03, box1_position_z), color=light_blue, wall_thickness=wall_thickness)

    robot = Robot(sm.SE3(robot_base_x, robot_base_y, robot_base_z))

    env.swift_env.add(robot.swift)
    env.swift_env.add(table1.swift)
    env.add_multiple_objects_swift(box_red.boxes)
    env.add_multiple_objects_swift(box_green.boxes)
    env.add_multiple_objects_swift(box_blue.boxes)
    env.swift_env.step(1)

    box1_center = sm.SE3(box1_position_x, box1_position_y - box_width - 0.03, box1_position_z + box_height/3)  # Lower and more centered
    box2_center = sm.SE3(box1_position_x, box1_position_y, box1_position_z + box_height/3)
    box3_center = sm.SE3(box1_position_x, box1_position_y + box_width + 0.03, box1_position_z + box_height/3)

    red_center_marker = sg.Sphere(radius=0.01, pose=box1_center, color=red)
    green_center_marker = sg.Sphere(radius=0.01, pose=box2_center, color=green)
    blue_center_marker = sg.Sphere(radius=0.01, pose=box3_center, color=blue)
    # env.swift_env.add(red_center_marker)
    # env.swift_env.add(green_center_marker)
    # env.swift_env.add(blue_center_marker)

    total_berries = 10

    def move_to_resolved_rate_of_motion(target_pose):
        errors_history = []
        while True:
            robot.move_to_target_resolved_rate_of_motion(target_pose)
            env.swift_env.step(robot.dt)
            current_error = np.linalg.norm(robot.error)
            errors_history.append(current_error)

            if robot.has_converged(errors_history):
                robot.reset_error()
                break
        


    for i in range(total_berries):
        berry = Berry(berry_radius, table_position_x, table_position_y, table_position_z, table_height, table_depth, table_width)
        env.swift_env.add(berry.swift)
        env.berries.append(berry)


    for i, berry in enumerate(env.berries):
        
        #phase 1:move directly to berry position
        target_pose = berry.pose
        move_to_resolved_rate_of_motion(target_pose)
        
        #phase 2:pick up berry
        robot.pick_berry(berry)
        time.sleep(0.2)

        #phase 3:flow directly to appropriate box
        drop_center = berry.get_drop_position_by_color(box1_center, box2_center, box3_center)
        move_pose = sm.SE3(drop_center.x, drop_center.y, drop_center.z + 0.1)
        move_to_resolved_rate_of_motion(move_pose)
        move_to_resolved_rate_of_motion(drop_center)

        #phase 4:drop berry in box
        robot.drop_berry_at(drop_center)
        if berry.random_color == red:
            env.swift_env.add(red_center_marker)
        else:
            if berry.random_color == green:
                env.swift_env.add(green_center_marker)
            else:
                env.swift_env.add(blue_center_marker)

        time.sleep(0.2)

        #phase 5:continue to next berry
        safe_pose = sm.SE3(drop_center.x, drop_center.y, drop_center.z + 0.15)
        move_to_resolved_rate_of_motion(safe_pose)


In [4]:
# resolved_rate_of_motion_sorting_task()